In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

try:
    cpus = mp.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

pool = mp.Pool(processes=cpus)

def next_training_batch(n):
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/l

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
W = tf.Variable(tf.truncated_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.truncated_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
y_image = tf.reshape(y, [-1, rows, cols, 1])

# First convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(y_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Third convolution layer
W_conv3 = weight_variable([5, 5, 64, 64])
b_conv3 = bias_variable([64])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

# Fully connected layer
W_fc1 = weight_variable([64, 1024])
b_fc1 = bias_variable([1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([1024, size*2])
b_fc2 = bias_variable([size*2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [10]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [11]:
# Loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y_conv))

In [12]:
# Counts the number of errors if we take the best guess for each sample
'''
def countErrors(xs,ys):
    predictions = sess.run(y_conv, feed_dict={mineCountsOneHot: batch_xs, keep_prob: 1.0})
    bestSquares = pool.map(np.argmax, predictions)
    unfrees = [(b == 0).astype(int) for b in batch_ys]
    frees = [unfrees[i][bestSquares[i]] for i in range(len(batch_xs))]
    return(sum(frees))
'''

'\ndef countErrors(xs,ys):\n    predictions = sess.run(y_conv, feed_dict={mineCountsOneHot: batch_xs, keep_prob: 1.0})\n    bestSquares = pool.map(np.argmax, predictions)\n    unfrees = [(b == 0).astype(int) for b in batch_ys]\n    frees = [unfrees[i][bestSquares[i]] for i in range(len(batch_xs))]\n    return(sum(frees))\n'

In [13]:
guesses = tf.diag_part(tf.gather(validGuessAverages,tf.argmax(y_conv,1), axis=1))
countErrors = tf.reduce_sum(tf.map_fn(lambda x: tf.cast(x < 1e-6, tf.int32), guesses, dtype=tf.int32))

In [14]:
# Summaries for tensorboard
_ = tf.summary.scalar('loss', cross_entropy)
_ = tf.summary.scalar('errors', countErrors)

In [15]:
# Optimiser
train_step = tf.train.AdamOptimizer(2e-5).minimize(cross_entropy)

In [16]:
savePathPrefix = '/media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/' + str(rows) + 'x' + str(cols) + '/'

modelDataPath = savePathPrefix

def summaryPath(run):
    return(savePathPrefix + 'runs/' + str(run) + '/')

In [17]:
# Create session and initialise or restore stuff
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(summaryPath(6), sess.graph)

In [18]:
tf.global_variables_initializer().run()

In [19]:
# Restore model?
saver.restore(sess, modelDataPath + "model-20610")

INFO:tensorflow:Restoring parameters from /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7560


In [20]:
# Train
for iteration in range(20611,1000001):
    batch_xs, batch_ys, _ = next_training_batch(10000)
    _ = sess.run(train_step, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 0.5})
    loss, summary = sess.run([cross_entropy, merged], feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 1.0})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 10 == 0:
        save_path = saver.save(sess, modelDataPath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-09 10:28:40: Loss at step 7561: 3.86653
2017-11-09 10:28:43: Loss at step 7562: 3.85413
2017-11-09 10:28:46: Loss at step 7563: 3.85576
2017-11-09 10:28:49: Loss at step 7564: 3.85369
2017-11-09 10:28:52: Loss at step 7565: 3.87002
2017-11-09 10:28:55: Loss at step 7566: 3.86873
2017-11-09 10:28:58: Loss at step 7567: 3.86021
2017-11-09 10:29:01: Loss at step 7568: 3.86378
2017-11-09 10:29:04: Loss at step 7569: 3.86553
2017-11-09 10:29:07: Loss at step 7570: 3.86101
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7570
2017-11-09 10:29:10: Loss at step 7571: 3.85366
2017-11-09 10:29:13: Loss at step 7572: 3.86425
2017-11-09 10:29:16: Loss at step 7573: 3.84475
2017-11-09 10:29:19: Loss at step 7574: 3.87209
2017-11-09 10:29:22: Loss at step 7575: 3.87615
2017-11-09 10:29:25: Loss at step 7576: 3.87922
2017-11-09 10:29:28: Loss at step 7577: 3.86109
2017-11-09 10:29:31: Loss at step 7578: 3.86845
2017-11-09 10:29:34: Loss at step 7579: 3.

2017-11-09 10:35:57: Loss at step 7705: 3.85649
2017-11-09 10:36:00: Loss at step 7706: 3.86515
2017-11-09 10:36:03: Loss at step 7707: 3.85583
2017-11-09 10:36:06: Loss at step 7708: 3.85776
2017-11-09 10:36:09: Loss at step 7709: 3.86108
2017-11-09 10:36:12: Loss at step 7710: 3.86758
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7710
2017-11-09 10:36:15: Loss at step 7711: 3.86843
2017-11-09 10:36:18: Loss at step 7712: 3.86383
2017-11-09 10:36:21: Loss at step 7713: 3.87319
2017-11-09 10:36:24: Loss at step 7714: 3.86164
2017-11-09 10:36:27: Loss at step 7715: 3.87191
2017-11-09 10:36:30: Loss at step 7716: 3.85447
2017-11-09 10:36:33: Loss at step 7717: 3.85998
2017-11-09 10:36:36: Loss at step 7718: 3.8563
2017-11-09 10:36:39: Loss at step 7719: 3.86273
2017-11-09 10:36:42: Loss at step 7720: 3.87139
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7720
2017-11-09 10:36:45: Loss at step 7721: 3.861

2017-11-09 10:43:10: Loss at step 7849: 3.85834
2017-11-09 10:43:13: Loss at step 7850: 3.86438
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7850
2017-11-09 10:43:16: Loss at step 7851: 3.86002
2017-11-09 10:43:19: Loss at step 7852: 3.86455
2017-11-09 10:43:22: Loss at step 7853: 3.85281
2017-11-09 10:43:25: Loss at step 7854: 3.85965
2017-11-09 10:43:28: Loss at step 7855: 3.86385
2017-11-09 10:43:31: Loss at step 7856: 3.86091
2017-11-09 10:43:34: Loss at step 7857: 3.86058
2017-11-09 10:43:37: Loss at step 7858: 3.86345
2017-11-09 10:43:40: Loss at step 7859: 3.85519
2017-11-09 10:43:43: Loss at step 7860: 3.85725
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-7860
2017-11-09 10:43:46: Loss at step 7861: 3.85989
2017-11-09 10:43:49: Loss at step 7862: 3.85006
2017-11-09 10:43:52: Loss at step 7863: 3.85548
2017-11-09 10:43:55: Loss at step 7864: 3.86074
2017-11-09 10:43:58: Loss at step 7865: 3.85

2017-11-09 10:50:18: Loss at step 7991: 3.86103
2017-11-09 10:50:21: Loss at step 7992: 3.86059
2017-11-09 10:50:24: Loss at step 7993: 3.86439
2017-11-09 10:50:27: Loss at step 7994: 3.86564
2017-11-09 10:50:30: Loss at step 7995: 3.86143
2017-11-09 10:50:33: Loss at step 7996: 3.87871
2017-11-09 10:50:36: Loss at step 7997: 3.86068
2017-11-09 10:50:39: Loss at step 7998: 3.86338
2017-11-09 10:50:42: Loss at step 7999: 3.85811
2017-11-09 10:50:45: Loss at step 8000: 3.86885
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-8000
2017-11-09 10:50:48: Loss at step 8001: 3.85381
2017-11-09 10:50:51: Loss at step 8002: 3.86781
2017-11-09 10:50:54: Loss at step 8003: 3.86333
2017-11-09 10:50:57: Loss at step 8004: 3.86751
2017-11-09 10:51:00: Loss at step 8005: 3.85391
2017-11-09 10:51:03: Loss at step 8006: 3.86464
2017-11-09 10:51:06: Loss at step 8007: 3.86588
2017-11-09 10:51:09: Loss at step 8008: 3.86338
2017-11-09 10:51:12: Loss at step 8009: 3.

2017-11-09 10:57:38: Loss at step 8135: 3.86115
2017-11-09 10:57:41: Loss at step 8136: 3.85279
2017-11-09 10:57:44: Loss at step 8137: 3.86139
2017-11-09 10:57:47: Loss at step 8138: 3.8636
2017-11-09 10:57:50: Loss at step 8139: 3.86304
2017-11-09 10:57:52: Loss at step 8140: 3.86493
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-8140
2017-11-09 10:57:56: Loss at step 8141: 3.86397
2017-11-09 10:57:59: Loss at step 8142: 3.86483
2017-11-09 10:58:01: Loss at step 8143: 3.8648
2017-11-09 10:58:04: Loss at step 8144: 3.86535
2017-11-09 10:58:07: Loss at step 8145: 3.8606
2017-11-09 10:58:10: Loss at step 8146: 3.87065
2017-11-09 10:58:13: Loss at step 8147: 3.87394
2017-11-09 10:58:16: Loss at step 8148: 3.86149
2017-11-09 10:58:19: Loss at step 8149: 3.8707
2017-11-09 10:58:22: Loss at step 8150: 3.86651
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-8150
2017-11-09 10:58:25: Loss at step 8151: 3.86025


...

2017-11-09 21:08:23: Loss at step 20151: 3.81102
2017-11-09 21:08:26: Loss at step 20152: 3.81387
2017-11-09 21:08:29: Loss at step 20153: 3.81111
2017-11-09 21:08:32: Loss at step 20154: 3.80506
2017-11-09 21:08:35: Loss at step 20155: 3.81261
2017-11-09 21:08:38: Loss at step 20156: 3.80535
2017-11-09 21:08:41: Loss at step 20157: 3.81221
2017-11-09 21:08:44: Loss at step 20158: 3.80984
2017-11-09 21:08:47: Loss at step 20159: 3.80925
2017-11-09 21:08:50: Loss at step 20160: 3.80909
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20160
2017-11-09 21:08:53: Loss at step 20161: 3.81579
2017-11-09 21:08:56: Loss at step 20162: 3.80944
2017-11-09 21:08:59: Loss at step 20163: 3.80736
2017-11-09 21:09:02: Loss at step 20164: 3.81137
2017-11-09 21:09:05: Loss at step 20165: 3.81977
2017-11-09 21:09:08: Loss at step 20166: 3.81534
2017-11-09 21:09:11: Loss at step 20167: 3.79866
2017-11-09 21:09:14: Loss at step 20168: 3.81752
2017-11-09 21:09:17: Lo

2017-11-09 21:15:26: Loss at step 20292: 3.80713
2017-11-09 21:15:29: Loss at step 20293: 3.81463
2017-11-09 21:15:32: Loss at step 20294: 3.82212
2017-11-09 21:15:35: Loss at step 20295: 3.80813
2017-11-09 21:15:38: Loss at step 20296: 3.80416
2017-11-09 21:15:41: Loss at step 20297: 3.81381
2017-11-09 21:15:44: Loss at step 20298: 3.8182
2017-11-09 21:15:47: Loss at step 20299: 3.80915
2017-11-09 21:15:50: Loss at step 20300: 3.80738
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20300
2017-11-09 21:15:53: Loss at step 20301: 3.81196
2017-11-09 21:15:56: Loss at step 20302: 3.82412
2017-11-09 21:15:59: Loss at step 20303: 3.80822
2017-11-09 21:16:02: Loss at step 20304: 3.81636
2017-11-09 21:16:04: Loss at step 20305: 3.82056
2017-11-09 21:16:08: Loss at step 20306: 3.818
2017-11-09 21:16:11: Loss at step 20307: 3.81222
2017-11-09 21:16:13: Loss at step 20308: 3.80656
2017-11-09 21:16:16: Loss at step 20309: 3.80978
2017-11-09 21:16:19: Loss 

2017-11-09 21:22:28: Loss at step 20433: 3.81628
2017-11-09 21:22:31: Loss at step 20434: 3.81263
2017-11-09 21:22:34: Loss at step 20435: 3.81898
2017-11-09 21:22:37: Loss at step 20436: 3.81788
2017-11-09 21:22:40: Loss at step 20437: 3.8267
2017-11-09 21:22:43: Loss at step 20438: 3.81615
2017-11-09 21:22:46: Loss at step 20439: 3.81086
2017-11-09 21:22:49: Loss at step 20440: 3.81977
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20440
2017-11-09 21:22:52: Loss at step 20441: 3.82218
2017-11-09 21:22:55: Loss at step 20442: 3.81118
2017-11-09 21:22:58: Loss at step 20443: 3.80954
2017-11-09 21:23:01: Loss at step 20444: 3.81758
2017-11-09 21:23:04: Loss at step 20445: 3.83004
2017-11-09 21:23:07: Loss at step 20446: 3.82776
2017-11-09 21:23:10: Loss at step 20447: 3.81143
2017-11-09 21:23:13: Loss at step 20448: 3.82122
2017-11-09 21:23:16: Loss at step 20449: 3.82556
2017-11-09 21:23:19: Loss at step 20450: 3.81269
Model saved in file: /me

2017-11-09 21:29:31: Loss at step 20574: 3.80805
2017-11-09 21:29:34: Loss at step 20575: 3.81426
2017-11-09 21:29:37: Loss at step 20576: 3.79974
2017-11-09 21:29:40: Loss at step 20577: 3.81173
2017-11-09 21:29:43: Loss at step 20578: 3.80836
2017-11-09 21:29:46: Loss at step 20579: 3.80709
2017-11-09 21:29:49: Loss at step 20580: 3.81317
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines7/8x8/model-20580
2017-11-09 21:29:52: Loss at step 20581: 3.81061
2017-11-09 21:29:55: Loss at step 20582: 3.81626
2017-11-09 21:29:58: Loss at step 20583: 3.81696
2017-11-09 21:30:01: Loss at step 20584: 3.82477
2017-11-09 21:30:04: Loss at step 20585: 3.80465
2017-11-09 21:30:07: Loss at step 20586: 3.81432
2017-11-09 21:30:10: Loss at step 20587: 3.81049
2017-11-09 21:30:13: Loss at step 20588: 3.80876
2017-11-09 21:30:17: Loss at step 20589: 3.80672
2017-11-09 21:30:20: Loss at step 20590: 3.80328
Model saved in file: /media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.

KeyboardInterrupt: 

In [ ]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(10000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 1.0}))

In [ ]:
# Run a test
batchSize = 10000
batch_xs, batch_ys,_ = next_training_batch(batchSize)

#countErrors(batch_xs,batch_ys)
len(batch_xs)

In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [ ]:
np.save("./W", sess.run(W))

In [ ]:
np.save("./b", sess.run(b))

In [ ]:
np.savez("./model", sess.run([W,b]))